In [ ]:
import numpy as np
import pandas as pd     
import re                                       

## datetime preprocessing

In [ ]:
data_types = {"id": "object", "author_id": "object"}
df = pd.read_csv("./tweets.csv", index_col=False, header=None, 
            names=['id', 'created_at', 'text', 'author_id', 'referenced_tweets'], dtype=data_types)

In [ ]:
# remove duplicate by column "id"
df = df.drop_duplicates(subset=['id'])

In [ ]:
# convert column "created_at" into date type
def clean_datetime(datetime):
    datetime = re.sub('.000Z', '', datetime)
    date = datetime.split('T')[0]
    return date
df["date"] = df["created_at"].apply(clean_datetime)
df['date'] =  pd.to_datetime(df['date'], format='%Y-%m-%d', errors='coerce') # NaT

In [ ]:
# select records from 2020-09-01 to 2020-12-01 and from 2021-02-01 to 2021-05-01 (6 months)
df_2020 = df[np.logical_and(df['date'] >= '2020-09-01', df['date'] < '2020-12-01')]
df_2021 = df[np.logical_and(df['date'] >= '2021-02-01', df['date'] < '2021-05-01')]
df = df_2020.append(df_2021, ignore_index=True)

In [ ]:
print(df.head())
print(df.shape)

In [ ]:
# df = df[['author_id', 'id', 'referenced_tweets', 'text', 'date']]
# df.to_csv("./tweets_01.csv", index=False, header=False)
data_types = {"id": "object", "author_id": "object"}
df = pd.read_csv("./tweets_01.csv", index_col=False, header=None, 
            names=['author_id', 'id', 'referenced_tweets', 'text', 'date'], dtype=data_types)


In [ ]:
df.head()

## tweets text preprocessing

In [ ]:
# remove unnesacery characters in the tweets text
def clean_text(text):    
    text = re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", "", text) # remove @mentions, '#' hashtag, hyperlink, punctuations, emojis and other special characters
    text = re.sub('RT', '', text) # remove RT
    # emoji_pattern = re.compile("["
    #                            u"\U0001F600-\U0001F64F"  # emoticons
    #                            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
    #                            u"\U0001F680-\U0001F6FF"  # transport & map symbols
    #                            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
    #                            u"\U00002500-\U00002BEF"  # chinese char
    #                            u"\U00002702-\U000027B0"
    #                            u"\U00002702-\U000027B0"
    #                            u"\U000024C2-\U0001F251"
    #                            u"\U0001f926-\U0001f937"
    #                            u"\U00010000-\U0010ffff"
    #                            u"\u2640-\u2642"
    #                            u"\u2600-\u2B55"
    #                            u"\u200d"
    #                            u"\u23cf"
    #                            u"\u23e9"
    #                            u"\u231a"
    #                            u"\ufe0f"  # dingbats
    #                            u"\u3030"
    #                            "]+", flags=re.UNICODE) 
    # text = emoji_pattern.sub(r'', text)  # remove emojis    
    return text
df["cleaned_text"] = df["text"].apply(clean_text)

In [ ]:
# remove stopwords in the tweets text
# !pip install textblob
# !pip install nltk
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from textblob import TextBlob

In [ ]:
stop = stopwords.words('english')

In [ ]:
df["stopped_text"] = df["cleaned_text"].apply(lambda x: " ".join(x.lower() for x in x.split())) # lower casing
df["stopped_text"] = df["stopped_text"].apply(lambda x: " ".join(x for x in x.split() if x not in stop)) # remove stopwords

In [ ]:
# lemmatize each word with its part-of-speech (POS) tag and keep ADJ., NOUN., and VERB.  
def lemmatize_with_postag(sentence):
    sent = TextBlob(sentence)
    tag_dict = {"J": 'a', # ADJ
                "N": 'n', # NOUN
                "V": 'v'} # VERB
                # "R": 'r'} # ADV
    words_and_tags = [(w, tag_dict.get(pos[0], 'NaN')) for w, pos in sent.tags]  # other POS tags: 'NaN'
    lemmatized_list = [wd.lemmatize(tag) for wd, tag in words_and_tags if tag != 'NaN']
    return " ".join(lemmatized_list)

In [ ]:
df["lem_text"] = df["stopped_text"].apply(lemmatize_with_postag)

In [ ]:
df_tweet = df

In [ ]:

df_tweet.head()

## merge two datasets

In [ ]:
data_types = {"author_id": "object"}
df_user = pd.read_csv("./users_01.csv", index_col=False, header=None, names=['verified', 'name', 'screen_name', 'img_url', 'author_id', 'description', 'location'], dtype=data_types)

In [ ]:
df_user.head()

In [ ]:
data_types = {"author_id": "object", "id": "object"}
df_tweets = pd.read_csv("./tweets_01.csv", index_col=False, header=None, names=['author_id', 'id', 'referenced_tweets', 'text', 'date', 'cleaned_text', 'stopped_text', 'lem_text'], dtype=data_types, parse_dates=["date"])

In [ ]:
df_tweets.head()

In [ ]:
df_all = df_tweets.merge(df_user, on="author_id", how='left')

In [ ]:
df_all.shape

## write into new files

In [ ]:
df = df_all[['author_id', 'id', 'referenced_tweets', 'text', 'date', 'cleaned_text',
       'stopped_text', 'lem_text', 'verified', 'name', 'screen_name',
       'img_url', 'description', 'location']]
df.to_csv("./data_all.csv", index=False, header=False)